In [28]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
waste_df = pd.read_csv("./resources/austin_waste_and_diversion.csv",engine='python', error_bad_lines=False, delimiter=',', skip_blank_lines=True, header=0)
waste_df.head()


Skipping line 535183: unexpected end of data


,ï»¿dropoff_site,load_id,load_time,load_type,load_weight,report_date,route_number,route_type
0,MRF,328118,2009-05-27 12:55:00,RECYCLING - SINGLE STREAM,4060.0,2009-05-27,DW1,RECYCLING - SINGLE STREAM
1,BRAKER SITE,308336,2008-12-16 10:55:00,SWEEPING,NaN,2008-12-16,NW10-R,STREET CLEANING-RESIDENTIAL
2,TDS - MRF,541514,2012-09-06 13:56:00,RECYCLING - SINGLE STREAM,5060.0,2012-09-06,RHBU20,RECYCLING - SINGLE STREAM
3,TDS LANDFILL,689059,2016-03-14 07:56:00,SWEEPING,3720.0,2016-03-14,DSS04,SWEEPER DUMPSITES
4,HORNSBY BEND,108272,2004-12-02 15:29:00,YARD TRIMMING,13840.0,2004-12-02,YH08,YARD TRIMMINGS


In [27]:
routes_df = pd.read_csv("./resources/GarbageRecycle2015RR.csv",engine='python', error_bad_lines=False, delimiter=',', skip_blank_lines=True, header=0)
routes_df.head()

,LANDFILL,the_geom,GARB_RT,GARB_DAY,GARB_SUP,SUPER_NUM,OP_TYPE,RT_OLD
0,TDS,MULTIPOLYGON (((-97.70618263474258 30.36741600...,PAH83,Thursday,Gilbert,697.0,Auto,NaN
1,TDS,MULTIPOLYGON (((-97.83236579358113 30.16212865...,PAM54,Monday,Castillo,500.0,Auto,PAM84
2,TDS,MULTIPOLYGON (((-97.72977104057442 30.19009265...,PAM71,Monday,Carnline,596.0,Auto,NaN
3,TDS,MULTIPOLYGON (((-97.65374103416144 30.26269210...,PAH54,Thursday,Castillo,500.0,Auto,PAH75
4,TDS,MULTIPOLYGON (((-97.64683377149026 30.38640132...,PAW71,Wednesday,Carnline,596.0,Auto,NaN


In [22]:
waste_df.dtypes

ï»¿dropoff_site     object
load_id              int64
load_time           object
load_type           object
load_weight        float64
report_date         object
route_number        object
route_type          object
dtype: object

In [17]:
waste_df.describe()

,load_id,load_weight
count,535181.000000,4.788040e+05
mean,421736.006940,1.158039e+04
std,198963.512162,7.800556e+03
min,101222.000000,-4.480000e+03
25%,245010.000000,5.440000e+03
50%,387215.000000,1.082000e+04
75%,608054.000000,1.656000e+04
max,750483.000000,1.562821e+06


In [18]:
waste_df.nunique()

ï»¿dropoff_site        33
load_id            535143
load_time          423045
load_type              18
load_weight          5116
report_date          4342
route_number         1787
route_type             37
dtype: int64

In [20]:
route_total = waste_df.groupby(["route_number"]).count()["route_type"]
route_total

route_number
0BM00            10
ABR01             3
AFD-FIREWISE      9
BKR-BR            9
BKR-YT           74
               ... 
YW12            223
YW13             59
YWNON1           28
YWNON5            3
ZILKER           51
Name: route_type, Length: 1787, dtype: int64

In [25]:
load_counts = waste_df.groupby(["load_type"]).count()["load_id"]
load_counts

load_type
BAGGED LITTER                      40
BRUSH                           30043
BULK                            29373
DEAD ANIMAL                      5109
GARBAGE COLLECTIONS            188255
LITTER                           1187
MATTRESS                            3
MIXED LITTER                     1223
MULCH                            1276
RECYCLED METAL                    613
RECYCLING - COMINGLE            29533
RECYCLING - PAPER               30490
RECYCLING - PLASTIC BAGS           38
RECYCLING - SINGLE STREAM       94336
SWEEPING                        68599
TIRES                            2237
YARD TRIMMING                   52814
YARD TRIMMING - X-MAS TREES        12
Name: load_id, dtype: int64

In [ ]:
# drop na

In [ ]:
# merge 

In [ ]:
# convert load time to date, drop report date because report date could vary from load time

In [ ]:
# figure out range of dates